# Amazon SageMaker 持ち込みPyTorchコンテナによる MNIST の学習（Studioノートブック用）

### 1.背景

MNISTは、手書き文字の分類に広く使用されているデータセットです。 70,000個のラベル付きの28x28ピクセルの手書き数字のグレースケール画像で構成されています。 データセットは、60,000個のトレーニング画像と10,000個のテスト画像に分割されます。 手書きの数字 0から9の合計10のクラスがあります。 

このチュートリアルでは、SageMaker のマネージドコンテナイメージ以外に、独自のコンテナイメージを持ち込める持ち込みのことを学習していただくために、持ち込みの独自コンテナを使用して SageMaker で MNIST モデルをトレーニングする方法を示します。

### 2.セットアップ

SageMaker セッションを作成し、設定を開始し
Dockerfileから、持ち込み用のコンテンナを作成していきます。
Dockerfileはcontainerディレクトリ配下に配置されていて、PyTorchコンテナにrequirements.txt(pip installするパッケージリスト)経由でSageMaker Training Toolkitをpipインストールするように記述されています。

SageMaker Training Toolkitは、どのDockerコンテナにも簡単に追加でき、SageMakerと互換性のあるモデルをトレーニングすることができるパッケージです。詳細は下記をご確認ください。
https://github.com/aws/sagemaker-training-toolkit

最後にdocker buildコマンドによりビルドしていきます。

In [ ]:
import boto3

Amazon SageMaker Studio ではDockerコマンドが環境上使えなくなっていますので
コンテナイメージのビルドの際はdockerコマンドの代わりにsm-docker build コマンドを使う必要があります。
sm-docker build を実行する際は、sm-docker build が裏側で AWS CodeBuild を使うようになっていてるため
StudioノートにアタッチしたIAMロールに以下の作業を追加で実施いただく必要があります

 1.”AmazonEC2ContainerRegistryPowerUser”の追加アタッチ

 2.信頼ポリシーの追加（後述）

 3.インラインポリシーの追加（後述）

sm-docker のインストール

In [ ]:
!pip install sagemaker-studio-image-build

Studioノートブックにアタッチされているロールに以下のコマンドの出力結果をコピーして追加の信頼関係を追加してください。

In [ ]:
!cat container/trust_relationships.json

Studioノートブックにアタッチされているロールに以下のコマンドの出力結果をコピーしてインラインポリシー（もしくはポリシーを別途作成して）アタッチしてください。

In [ ]:
!cat container/inline_policy.json

### 3.Amazon ECRリポジトリの作成とDockerImageの登録

sm-docker buildコマンドでAmazon ECRにsagemaker-toolkit-container-pytorchというレポジトリを作成し、Studioノートブックに作成したDockerImageをpushしていきます。

sm-docker buildコマンドを実行するとrepositoryオプションに指定したレポジトリとタグ名でAmazon ECRにイメージがpushされます。

詳細は下記をご確認ください
https://pypi.org/project/sagemaker-studio-image-build/

In [ ]:
%%time

IMAGE_NAME = 'toolkit-container-pytorch'
TAG=':1.11'
REGION = boto3.session.Session().region_name

MY_ACCOUNT_ID = boto3.client('sts').get_caller_identity().get('Account')
MY_ECR_ENDPOINT = f'{MY_ACCOUNT_ID}.dkr.ecr.{REGION}.amazonaws.com/'

MY_REPOSITORY_NAME = f'sagemaker-{IMAGE_NAME}'
MY_REPOSITORY_URI = f'{MY_ECR_ENDPOINT}sagemaker-{IMAGE_NAME}'
MY_IMAGE_URI = f'{MY_REPOSITORY_URI}{TAG}'

%cd ./container
!sm-docker build . --repository {MY_REPOSITORY_NAME}{TAG}
%cd ../

print(f'コンテナイメージは {MY_IMAGE_URI} へ登録されています。')

### 4.セットアップ

学習およびモデルデータに使用する S3 バケットとプレフィックスは、ノートブックインスタンス、トレーニング、およびホスティングと同じリージョン内にある必要があります。

データへの学習およびホスティングアクセスを提供するために使用される IAM ロール arn を用います。 ノートブックインスタンス、学習インスタンス、および/またはホスティングインスタンスに複数のロールが必要な場合は、 sagemaker.get_execution_role（） を、適切な IAM ロール arn 文字列に置き換えてください。

In [ ]:
import sagemaker
from sagemaker.pytorch import PyTorch
bucket = sagemaker.session.Session().default_bucket()
prefix = 'sagemaker/DEMO-pytorch-mnist-byoc'
role = sagemaker.get_execution_role()

### 5.データの取得

In [ ]:
!aws s3 cp s3://fast-ai-imageclas/mnist_png.tgz . --no-sign-request
!tar -xzf mnist_png.tgz -C /opt/ml --no-same-owner

In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch
import os

root_dir_studio = '/opt/ml'
data_dir = os.path.join(root_dir_studio,'data') 
training_dir = os.path.join(root_dir_studio,'mnist_png/training') 
test_dir = os.path.join(root_dir_studio,'mnist_png/testing') 

os.makedirs(data_dir, exist_ok=True)

training_data = datasets.ImageFolder(root=training_dir,
                            transform=transforms.Compose([
                            transforms.Grayscale(),
                            transforms.ToTensor(),
                            transforms.Normalize((0.1307,), (0.3081,))]))
test_data = datasets.ImageFolder(root=test_dir,
                            transform=transforms.Compose([
                            transforms.Grayscale(),
                            transforms.ToTensor(),
                            transforms.Normalize((0.1307,), (0.3081,))]))

training_data_loader = DataLoader(training_data, batch_size=len(training_data))
training_data_loaded = next(iter(training_data_loader))
torch.save(training_data_loaded, os.path.join(data_dir, 'training.pt'))

test_data_loader = DataLoader(test_data, batch_size=len(test_data))
test_data_loaded = next(iter(test_data_loader))
torch.save(test_data_loaded, os.path.join(data_dir, 'test.pt'))

### 6.データをS3にアップロードする

データセットを S3 にアップロードするには、 sagemaker.Session.upload_data 関数を使用します。 戻り値として入力した S3 のロケーションは、後で学習ジョブを実行するときに使用します。

In [ ]:
inputs = sagemaker.session.Session().upload_data(path=data_dir, bucket=bucket, key_prefix=prefix)
print('input spec (in this case, just an S3 path): {}'.format(inputs))

### 7.学習を開始する

学習の条件を設定するため、Estimator クラスの子クラスの PyTorch オブジェクトを作成します。 ここでは、PyTorchスクリプト、IAMロール、および（ジョブごとの）ハードウェア構成を渡す PyTorch Estimator を定義しています。また合わせてentry_point を指定することで、依存するスクリプト群をコンテナにコピーして、学習時に使用することが可能です。

トレーニングジョブを開始する際、Amazon S3のどこにソースが格納されているか、起動するモジュールは何かをsagemaker-training-toolkitライブラリに知らせる必要があります。今回はPython SDKのフレームワーク使用していますので自動的に設定されています。 Estimator Classを利用する際は必要になりますのでご注意ください。

最後に、Amazon SageMaker Python SDKで定義されている汎用Estimatorオブジェクトのfit()メソッドを呼び出すことで、学習ジョブを実行することができます

In [ ]:
estimator = PyTorch(
    entry_point='train.py',
    image_uri=MY_IMAGE_URI,
    role=sagemaker.get_execution_role(),
    hyperparameters={
        'lr':0.01,
        'batch-size':16
    },
    instance_count=1,
    instance_type='ml.g4dn.xlarge',
)
estimator.fit({'training': inputs})

#Estimatorクラスで学習する場合
#estimator = sagemaker.estimator.Estimator(
#    image_uri=MY_IMAGE_URI,
#    role=sagemaker.get_execution_role(),
#    hyperparameters={
#        'lr':0.01,
#        'batch-size':16,
#        'sagemaker_program' : 'train.py',
#        'sagemaker_submit_directory' : 's3://'+bucket+'/'+estimator._current_job_name+'/source/sourcedir.tar.gz'
#    },
#    instance_count=1,
#    instance_type='ml.g4dn.xlarge',
#)
#estimator.fit({'training': inputs})